In [2]:
import numpy as np
import cv2
import sys
import os

In [1]:
import numpy as np
import cv2
from scipy import ndimage

def gen_xyz(fov, u, v, out_h, out_w):
    out = np.ones((out_h, out_w, 3), np.float32)

    x_rng = np.linspace(-np.tan(fov / 2), np.tan(fov / 2), num=out_w, dtype=np.float32)
    y_rng = np.linspace(-np.tan(fov / 2), np.tan(fov / 2), num=out_h, dtype=np.float32)

    out[:, :, :2] = np.stack(np.meshgrid(x_rng, -y_rng), -1)
    Rx = np.array([[1, 0, 0], [0, np.cos(v), -np.sin(v)], [0, np.sin(v), np.cos(v)]])
    Ry = np.array([[np.cos(u), 0, np.sin(u)], [0, 1, 0], [-np.sin(u), 0, np.cos(u)]])

    R = np.dot(Ry, Rx)
    return out.dot(R.T)

def xyz_to_uv(xyz):
    x, y, z = np.split(xyz, 3, axis=-1)
    u = np.arctan2(x, z)
    c = np.sqrt(x ** 2 + z ** 2)
    v = np.arctan2(y, c)
    return np.concatenate([u, v], axis=-1)

def uv_to_XY(uv, eq_h, eq_w):
    u, v = np.split(uv, 2, axis=-1)
    X = (u / (2 * np.pi) + 0.5) * eq_w - 0.5
    Y = (-v / np.pi + 0.5) * eq_h - 0.5
    return np.concatenate([X, Y], axis=-1)

def eq_to_pers(eqimg, fov, u, v, out_h, out_w):
    xyz = gen_xyz(fov, u, v, out_h, out_w)
    uv  = xyz_to_uv(xyz)

    eq_h, eq_w = eqimg.shape[:2]
    XY = uv_to_XY(uv, eq_h, eq_w)

    X, Y = np.split(XY, 2, axis=-1)
    X = np.reshape(X, (out_h, out_w))
    Y = np.reshape(Y, (out_h, out_w))

    mc0 = ndimage.map_coordinates(eqimg[:, :, 0], [Y, X]) # channel: B
    mc1 = ndimage.map_coordinates(eqimg[:, :, 1], [Y, X]) # channel: G
    mc2 = ndimage.map_coordinates(eqimg[:, :, 2], [Y, X]) # channel: R

    output = np.stack([mc0, mc1, mc2], axis=-1)
    return output


In [8]:

#import eq2cm
#sample.py images/589271383.507201.JPG output/

if __name__ == '__main__':
    args = sys.argv
    #img_path = args[1]
    #out_dir  = args[2]
    img_path = r"images/pict1.JPG"
    out_dir  = r"output/"
    eqimg = cv2.imread(img_path)

    for i in range(-4, 4, 2):
        print('processing...')
        pers = eq_to_pers(eqimg, np.pi/2, i * np.pi/4, 0, 480, 480)
        cv2.imwrite(os.path.join(out_dir, 'angles' + str(i + 4) + '.JPG'), pers)
        if(i+4==0):
            combined_image= pers
        else:
            combined_image = cv2.hconcat([combined_image,pers])
    cv2.imwrite(os.path.join(out_dir, 'angleconb' + '.JPG'), combined_image)

processing...
processing...
processing...
processing...


In [11]:
#テストコード
if __name__ == '__main__':
    cap = cv2.VideoCapture('images/動画.MP4') #読み込む動画のパス
    fps = cap.get(cv2.CAP_PROP_FPS)

    fourcc = cv2.VideoWriter_fourcc('m','p','4', 'v') #mp4フォーマット
    video = cv2.VideoWriter('output/edited_video.mp4', fourcc, fps, (480*4,480)) #書き込み先のパス、フォーマット、fps、サイズ
    
    while True:
        print('processing...')
        # 1フレームずつ取得する。
        ret, frame = cap.read()
        if not ret:
            break

    #🐥 任意の処理をここに書く 🐥
        for i in range(-4, 4, 2):
            
            pers = eq_to_pers(frame, np.pi/2, i * np.pi/4, 0, 480, 480)
            if(i+4==0):
                combined_image= pers
            else:
                combined_image = cv2.hconcat([combined_image,pers])
        
        cv2.imwrite(os.path.join(out_dir, 'angleconb' + '.JPG'), combined_image)

        video.write(combined_image)

        key = cv2.waitKey(30)
        if key == 27:
            break

    cap.release()
    video.release()
    print('Finished')



processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
proces